# UHF SCF

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## Molecular setup

UHF SCF calculations start off in the same way as RHF SCF calculations. We first need a second-quantized Hamiltonian that expresses all the integrals in the AO basis.

In [2]:
molecule = gqcpy.Molecule.HRingFromDistance(3, 1.889, 0)  # create a neutral molecule
N = molecule.numberOfElectrons()
N_alpha = N//2 + 1
N_beta = N//2

spinor_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
S = spinor_basis.quantizeOverlapOperator()

sq_hamiltonian = gqcpy.USQHamiltonian_d.Molecular(spinor_basis, molecule)  # 'sq' for 'second-quantized'

In [3]:
print(molecule)

Number of electrons: 3 
H  (1.09061, 0, 0)
H  (-0.545307, 0.9445, 0)
H  (-0.545307, -0.9445, 0)



## Plain UHF SCF calculations

Then, we need an UHF SCF solver and an UHF SCF environment.

In [4]:
environment = gqcpy.UHFSCFEnvironment_d.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)
plain_solver = gqcpy.UHFSCFSolver_d.Plain(threshold=1.0e-04, maximum_number_of_iterations=1000)  # the system is not converging very rapidly

Using GQCP's capabilities of 'injecting' Python code inside the C++ library, we'll add an intermediary step that doesn't modify the environment, but just prints out the current Fock matrices.

This is done by writing a `Python` function that manipulates the solver's environment (of type `EnvironmentType`), and subsequently wrapping that function into a `FunctionalStep_EnvironmentType`. The instance of type `FunctionalStep_EnvironmentType` is then subsequently inserted inside the algorithm that should be performed.

In this example, we'll print out the alpha- and beta- Fock matrices. The Fock matrices are stored as a list of unrestricted Fock-operators. Hence, we have to get alpha/beta component of the last entry and print its parameters.

In [5]:
def print_fock_matrices(environment):
    if environment.fock_matrices:
        print("Alpha Fock matrix:\n{}\n".format(environment.fock_matrices[-1].alpha.parameters()))
        print("Beta Fock matrix:\n{}".format(environment.fock_matrices[-1].beta.parameters()))
    else:
        print("No Fock matrices in the queue.")

print_step = gqcpy.FunctionalStep_UHFSCFEnvironment_d(print_fock_matrices)

In [6]:
plain_solver.insert(print_step, index=0)  # insert at the front

We can use the solver to optimize the UHF parameters.

In [7]:
qc_structure = gqcpy.UHF_d.optimize(plain_solver, environment)

No Fock matrices in the queue.
Alpha Fock matrix:
[[-0.488 -0.453 -0.453]
 [-0.453 -0.381 -0.56 ]
 [-0.453 -0.56  -0.381]]

Beta Fock matrix:
[[-0.083 -0.444 -0.444]
 [-0.444 -0.257 -0.474]
 [-0.444 -0.474 -0.257]]
Alpha Fock matrix:
[[-0.515 -0.456 -0.456]
 [-0.456 -0.368 -0.555]
 [-0.456 -0.555 -0.368]]

Beta Fock matrix:
[[-0.049 -0.434 -0.434]
 [-0.434 -0.272 -0.489]
 [-0.434 -0.489 -0.272]]
Alpha Fock matrix:
[[-0.525 -0.457 -0.457]
 [-0.457 -0.363 -0.553]
 [-0.457 -0.553 -0.363]]

Beta Fock matrix:
[[-0.036 -0.429 -0.429]
 [-0.429 -0.276 -0.493]
 [-0.429 -0.493 -0.276]]
Alpha Fock matrix:
[[-0.528 -0.458 -0.458]
 [-0.458 -0.361 -0.552]
 [-0.458 -0.552 -0.361]]

Beta Fock matrix:
[[-0.031 -0.427 -0.427]
 [-0.427 -0.278 -0.495]
 [-0.427 -0.495 -0.278]]
Alpha Fock matrix:
[[-0.529 -0.458 -0.458]
 [-0.458 -0.361 -0.552]
 [-0.458 -0.552 -0.361]]

Beta Fock matrix:
[[-0.03  -0.426 -0.426]
 [-0.426 -0.278 -0.495]
 [-0.426 -0.495 -0.278]]
Alpha Fock matrix:
[[-0.53  -0.458 -0.458]
 [-0.4

In [8]:
print(qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

-1.328057162456922


In [9]:
uhf_parameters = qc_structure.groundStateParameters()

In [10]:
print(uhf_parameters.orbitalEnergies().alpha)

[-0.702 -0.091  0.38 ]


In [11]:
print(uhf_parameters.orbitalEnergies().beta)

[-0.568  0.432  0.596]


In [12]:
print(plain_solver.numberOfIterations())

10


## DIIS UHF SCF calculations

In [13]:
environment = gqcpy.UHFSCFEnvironment_d.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)
diis_solver = gqcpy.UHFSCFSolver_d.DIIS(threshold=1.0e-04, maximum_number_of_iterations=1000, minimum_subspace_dimension=6, maximum_subspace_dimension=6)  # the system is not converging very rapidly

In [14]:
qc_structure = gqcpy.UHF_d.optimize(diis_solver, environment)

In [15]:
print(qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

-1.3280571497657099


In [16]:
uhf_parameters = qc_structure.groundStateParameters()

In [17]:
print(uhf_parameters.orbitalEnergies().alpha)

[-0.702 -0.091  0.38 ]


In [18]:
print(uhf_parameters.orbitalEnergies().beta)

[-0.568  0.432  0.596]


In [19]:
print(diis_solver.numberOfIterations())

10


## Complex UHF SCF calculations

The same approach can be used for complex UHF calculations. We just replace the "_d" with "_cd" indicating the use of complex doubles.

In [21]:
complex_spinor_basis = gqcpy.USpinOrbitalBasis_cd(molecule, "STO-3G")
complex_S = complex_spinor_basis.quantizeOverlapOperator()

complex_sq_hamiltonian = gqcpy.USQHamiltonian_cd.Molecular(complex_spinor_basis, molecule)  # 'sq' for 'second-quantized'

In [22]:
complex_environment = gqcpy.UHFSCFEnvironment_cd.WithComplexlyTransformedCoreGuess(N_alpha, N_beta, complex_sq_hamiltonian, complex_S)
complex_plain_solver = gqcpy.UHFSCFSolver_cd.Plain(threshold=1.0e-04, maximum_number_of_iterations=1000)  # the system is not converging very rapidly

In [23]:
complex_qc_structure = gqcpy.UHF_cd.optimize(complex_plain_solver, complex_environment)

In [24]:
print(complex_qc_structure.groundStateEnergy().real + gqcpy.Operator.NuclearRepulsion(molecule).value())

-1.3358471551411335
